## Creating An AI-Based JFK Speech Writer: Part 2
-----------------------

__[1. Introduction](#first-bullet)__

__[2. Data Preparation](#second-bullet)__

__[3. A Bidirectional GRU Model](#third-bullet)__

__[4. Generating Text](#fourth-bullet)__

__[5. Next Steps](#fifth-bullet)__


## Introduction <a class="anchor" id="first-bullet"></a>
----

In this blog post I follow up on the last [post](http://michael-harmon.com/blog/jfk1.html) and develop a model for text generation using [Recurrent Neural Networks](https://en.wikipedia.org/wiki/Recurrent_neural_network). I'll build a bi-directional [gated recurrent unit (GRU)](https://en.wikipedia.org/wiki/Gated_recurrent_unit) that is trained on speeches made by [President John F. Kennedy](https://en.wikipedia.org/wiki/John_F._Kennedy). Specifically, I'll go over how to build a model that predicts the "next word" in a sentence based off a sequence of the words coming before it. This project was more challenging than I initially anticipated due to the data preparation needs of the problem. The data preparation was more involved then other posts that I have done on natural language processing since it involves modeling a sequences of words instead of using a "[bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model)." I'll go over some of these details more in the post.
 
The concept of sequence modeling in recurrent neural networks is also different from other models that I have done in the past and I will spend some time covering this topic. Interestingly, the next word prediction turns out to be a multi-class classification problem, albeit with a very large number of classes! Let's get started with the problem. 

The first step is to import the necessary [TensorFlow](https://www.tensorflow.org/) and [Google Cloud](https://www.tensorflow.org/) Python packages (since the data is in [Google Cloud Storage](https://cloud.google.com/storage?)) :

In [1]:
import numpy as np
import tensorflow as tf 

from google.oauth2 import service_account
from google.cloud import storage

tf.compat.v1.logging.set_verbosity('ERROR')
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Data Preparation <a class="anchor" id="second-bullet"></a>
----

I next connect to [Google Cloud Storage](https://cloud.google.com/storage?) to download all the concatenated speeches by President Kennedy. To do this I get my GCP credentials and then instantiate the client to connect to the bucket `gs://harmon-kennedy/`

In [2]:
credentials = service_account.Credentials.from_service_account_file('credentials.json')
client = storage.Client(project=credentials.project_id, credentials=credentials)

In [3]:
bucket = client.get_bucket("harmon-kennedy")

Now I can download all the speeches that were concatenated into one file,

In [4]:
blob = bucket.blob("all_jfk_speeches.txt")
text = blob.download_as_text()

I can see the first 300 characters of the text are,

In [5]:
text[:300]

'Of particular importance to South Dakota are the farm policies of the Republican party - the party of Benson, Nixon and Mundt - the party which offers our young people no incentive to return to the farm - which offers the farmer only the prospect of lower and lower income - and which offers the nati'

To get situated with the data I can get the number of characters in the text as well as the number of unique characters,

In [6]:
print(f'Length of text: {len(text)} characters')

Length of text: 7734579 characters


In [7]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

67 unique characters


Since I'll be making a word level model this isn't totally helpful. Instead I'll get the total number of words and number of unique words. To do this I need to clean the text; convert newline characters to spaces, remove non-English characters and convert characters to lower case.

In [10]:
words = text.replace("\n", " ").split(" ")

In [11]:
clean_words = [word.lower() for word in words if word.isalpha()]

In [12]:
clean_text = " ".join(clean_words)

The impact this had on the same text from above can be seen below,

In [13]:
clean_text[:300]

'of particular importance to south dakota are the farm policies of the republican party the party of nixon and mundt the party which offers our young people no incentive to return to the farm which offers the farmer only the prospect of lower and lower income and which offers the nation the vision of'

The total number of clean words and unique clean words in the text are,

In [14]:
print(f"{len(clean_words)} number of clean words")

1196835 number of clean words


In [15]:
print(f"{len(set(clean_words))} unique clean words")

19291 unique clean words


Now let's talk about how we can process out text data for training a model to predict the next word.

The way a word level text generation model is built is to take a sequence of N words and then predict the next one. To create a training set, the text is split up into sliding widows where the feature vector **x** is the N words in the sequence of text and the target y is the N+1 word in that text. We repeat this process for N=1,2,3,4,... 

For instance take the sentence "the man is walking down the street." To build a model that predicts the next word based on the 4 word that come before it, it is necessary to create the 4 training examples as shown below,

<figure>
<img src="images/nextword.png" alt="Trulli" style="width:75%">
<figcaption align = "center">
From https://www.youtube.com/watch?v=VAMKuRAh2nc
</figcaption>
</figure>

For this model I will use `seq_length` as `N` or the number words in the text used to predict the next word. In order to be able to predict the next word I need to reduce the total number words that are possible to predict to a finite number. This means limiting the number of possible words to be a set of size `vocab_size`. This in turn converts the next word prediction problem into a classification problem with `vocab-size` classes.

In order to convert the text which is represented as a sequence of words into numerical vectors I'll use the [TextVectorization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization) class. This technique is discussed in more in a prior post which you can read [here](http://michael-harmon.com/blog/NLP4.html).

In [17]:
vocab_size = 15000
seq_length = 50

I first instantiate the `TextVectorization` layer and fit it to the text:

In [19]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer_layer = TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=seq_length,
)

Metal device set to: Apple M1


2023-06-01 19:08:57.300905: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-01 19:08:57.301354: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [20]:
vectorizer_layer.adapt([clean_text])

2023-06-01 19:08:58.454915: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-01 19:08:58.502828: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Note that I do this on the `clean_text` string and not the text string. 

I can then get the set of words in the `vectorizer_layer`'s "vocabulary" and create a dictionary to look up each word's equivalent numerical value.

In [21]:
voc = vectorizer_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

We can see the vocab size of the `vectorizer_layer`,

In [23]:
len(voc)

15000

The numerical value for each of the first two words in the example text above is then,

In [24]:
word_index['of']

3

In [25]:
word_index['particular']

717

The numerical value for the "out of vocabulary" token is,

In [26]:
word_index['[UNK]']

1

Next I'll create the dataset X and y, where X is the vector of features, which in turn are numerical values for the sequence of words. The vector y is the target which is integer that represents the numerical value of next word in that sequence:

In [27]:
words_seq = [clean_words[i:i + seq_length] for i in range(0, len(clean_words) - seq_length-1)]
next_word = [clean_words[i + seq_length] for i in range(0, len(clean_words) - seq_length-1)]

Each entry in `words_seq` is a list of the `seq_length` words or tokens that make that sequence in that training example. 

In [30]:
for words in words_seq[:2]:
    print(" ".join(words) + "\n")

of particular importance to south dakota are the farm policies of the republican party the party of nixon and mundt the party which offers our young people no incentive to return to the farm which offers the farmer only the prospect of lower and lower income and which offers the

particular importance to south dakota are the farm policies of the republican party the party of nixon and mundt the party which offers our young people no incentive to return to the farm which offers the farmer only the prospect of lower and lower income and which offers the nation



Now I'll convert the target vector of "next words" to a vector with "numerical values" using the `word_index` dictionary:

In [31]:
next_cat = np.array([word_index.get(word, 1) for word in next_word])
next_cat[:2]

array([111,   2])

Notice that if the word is not in the `word_index` then it is given the out of vocabulary int of 1.

Then I convert those list of lists into a list of strings,

In [32]:
X = np.array([" ".join(words_seq[i]) for i in range(len(next_word))
              if next_cat[i] != 1]).reshape(-1,1)
X[:2]

array([['of particular importance to south dakota are the farm policies of the republican party the party of nixon and mundt the party which offers our young people no incentive to return to the farm which offers the farmer only the prospect of lower and lower income and which offers the'],
       ['particular importance to south dakota are the farm policies of the republican party the party of nixon and mundt the party which offers our young people no incentive to return to the farm which offers the farmer only the prospect of lower and lower income and which offers the nation']],
      dtype='<U406')

The reason for doing this is that this way my model will be able to take inputs that are just plain text instead of needing lists of strings that represent that text. The later would require that new inputs to the model be pre-processed before being feed into the trained model, while the latter means a trained model can just take raw text as the input.


Notice that I only included sequences of the text where the target word was not an out of vocabulary word. 

The next two words that correspond to the targets for the examples above are,

In [167]:
next_word[:2]

['from', 'this']

Lastly, I'll create the target vector by filtering out the case where value would be out-of-vocabulary tokens:

In [33]:
y = np.array([cat for cat in next_cat if cat != 1])
y[:2]

array([111,   2])

The reason for filtering the out-of-vocabulary tokens is I don't want to train a model that predicts out-of-vocabulary words since this would be meaningless to end users.

The size of each of the feature dataset,

In [34]:
X.shape

(1192491, 1)

That is X is technically a 1-D array, but each entry in X is an array that contains the text. Once we transform the X array we will have a matrix it will be of size,

In [37]:
vectorizer_layer.call(X).shape

TensorShape([1192491, 50])

This is what we would expect, 50 features per entry in our design matrix. Again, I use this set up where X is a 1 dimensional array so that my model has only input of text.

The target variable has shape,

In [38]:
y.shape

(1192491,)

Now to see what effect the vectorizer layer has on the text I'll feed the first two sequences above through the layer.

In [39]:
vectorizer_layer.call(X[:2])

<tf.Tensor: shape=(2, 50), dtype=int64, numpy=
array([[   3,  717,  652,    5,  482, 2772,   16,    2,  143,  280,    3,
           2,  142,   81,    2,   81,    3,  192,    4, 8230,    2,   81,
          23, 1290,   13,  406,   57,   46, 3001,    5,  978,    5,    2,
         143,   23, 1290,    2,  431,   60,    2, 2429,    3,  913,    4,
         913,  285,    4,   23, 1290,    2],
       [ 717,  652,    5,  482, 2772,   16,    2,  143,  280,    3,    2,
         142,   81,    2,   81,    3,  192,    4, 8230,    2,   81,   23,
        1290,   13,  406,   57,   46, 3001,    5,  978,    5,    2,  143,
          23, 1290,    2,  431,   60,    2, 2429,    3,  913,    4,  913,
         285,    4,   23, 1290,    2,  111]])>

The vectorizer layer converts the array of strings with shape `(1179990,)` to an matrix of integers of shape `(1179990, seq_length)`. Each entry in the array will be a integer from 1 to `vocab_size` and is the integer representation for each word.

Now that we an understanding of how we can create the dataset let's talk about Recurrent Neural Networks. 

## A Bidirectional GRU Model  <a class="anchor" id="third-bullet"></a>
------------
[Recurrent Neural Networks (RNN)](https://en.wikipedia.org/wiki/Recurrent_neural_network) are deep learning models used to predict sequences. These models use an internal state, **h**, to act as memory that processes these sequences and "remember" things from the past. A quintessential diagram of a RNN is shown below, 


<figure>
<img src="images/rnn.svg" alt="Trulli" style="width:75%">
<figcaption align = "center">
From https://en.wikipedia.org/wiki/Recurrent_neural_network#/media/File:Recurrent_neural_network_unfold.svg/
</figcaption>
</figure>

A RNN cell is shown on the left and on the right is the "un-rolled" version that shows how the cell processes a sequence of inputs **x** into outputs **o**; there is a subscript *t* that denotes entry in the sequence. The subscript for each **h** is used to denote the value the internal state or memory cell in the t-th entry in the sequence.

There are a number of RNN's and a few are shown below,

<figure>
<img src="images/types.png" alt="Trulli" style="width:75%">
<figcaption align = "center">
From https://calvinfeng.gitbook.io/machine-learning-notebook/supervised-learning/recurrent-neural-network/recurrent_neural_networks/
</figcaption>
</figure>


The model I am building in this post that uses a sequence of words to predict the next word is a "many-to-one" model. The many-to-one RNN gets its name since we using a sequence "many" of word to predict one word, i.e. the next word.  Zooming into the RNN cell we focus on a specific type of RNN called a [Gated Recurrent Unit (GRU)](https://en.wikipedia.org/wiki/Gated_recurrent_unit). The details of a GRU cell are shown below.


<figure>
<img src="images/gru.png" alt="Trulli" style="width:75%">
<figcaption align = "center">
From https://colah.github.io/posts/2015-08-Understanding-LSTMs/
</figcaption>
</figure>

There is a hidden state **h** that takes on values for each iteration *t*. There is a candidate update to the hidden state **h** with a ~ over it. The candidate update to the hidden state has values between -1 and +1 and is a function of the relevance gate **r** as well as the prior value of the hidden state and the current value of the input. The relevance gate is value between 0 and 1 and is a function of the prior value of the hidden state and the current value of the input. It controls the amount off effect that the prior hidden state value has on the candidate update value for the hidden state. 

Lastly, there is a forget gate **z** which is between 0 and 1 is a function of the prior value of the hidden state and the current value of the input. The forget gate is used to control whether we update the hidden state value or not. If `z = 1` then we update the internal state to be the candidate state. If `z = 0`, the value for the hidden state remains unchanged.

Notice the hidden state value **h** of one iteration can be fed directly into the RNN as well as the input **x**. These variables are not necessarily scalars and are often vectors. 

In the model I am building the variables will be vectors of dimension `seq_length`. The output of the RNN cell is a vector of size `vocab_size`. To convert the hidden state vector **x** to **y** we apply a softmax function.

Many times in natural language processing models make use of a [bi-directional RNN](https://en.wikipedia.org/wiki/Bidirectional_recurrent_neural_networks). In this type of model two RNN cells are used, one processing the sequence in the forward direction and one processing the sequence in the reverse direction. The architecture is shown below:

<figure>
<img src="images/bidirectionalgru.png" alt="Trulli" style="width:75%">
<figcaption align = "center">
From https://www.researchgate.net/figure/The-structure-of-a-bidirectional-GRU-model_fig4_366204325
</figcaption>
</figure>

Notice that the forward and backward GRU cells are both a function of the same input value **x** (both at the same time *t*), but are functions of different iterations hidden states **h** (different values of *t*. Both cells at the same iteration are used to compute the output at the same iteration. Bidirectional RNN's were introduced to increase the amount of input information available to the network.

I create a bi-directional GRU model using [TensorFlow's subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models) methods below.

In [ ]:
from typing import Dict

class JFKSpeechWriter(tf.keras.Model):
    def __init__(self, 
                 text: str, 
                 seq_length: int,
                 vocab_size: int, 
                 embedding_dim: int, 
                 units: int) -> None:
        
        super().__init__()
        
        self.vectorizer_layer = TextVectorization(
                                    standardize="lower_and_strip_punctuation",
                                    max_tokens=vocab_size,
                                    output_mode="int",
                                    output_sequence_length=seq_length,
                              )
        self.embedding_layer =  tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.GRU_layer = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units))
        self.dense_layer = tf.keras.layers.Dense(vocab_size, activation='softmax')
        
        self.vectorizer_layer.adapt([text])
        
    def call(self, input_str: str) -> int:
        # x = self.input_layer(input_str)
        x = self.vectorizer_layer(input_str)
        x = self.embedding_layer(x)
        x = self.GRU_layer(x)
        return self.dense_layer(x)
        
    def get_wordmap(self) -> Dict[int, str]:
        voc = self.vectorizer_layer.get_vocabulary()
        word_index = dict(zip(voc, range(len(voc))))
        return dict(map(reversed, word_index.items()))


2023-05-22 22:06:00.820847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [32]:
def build_model(
             text: str, 
             seq_length: int,
             vocab_size: int, 
             embedding_dim: int, 
             units: int
) -> tf.keras.models.Sequential:
    
    vectorizer_layer = TextVectorization(
                            standardize="lower_and_strip_punctuation",
                            max_tokens=vocab_size,
                            output_mode="int",
                            output_sequence_length=seq_length)
    
    vectorizer_layer.adapt([text])
    
    model = tf.keras.models.Sequential([
                    tf.keras.Input(shape=(1,), 
                                   dtype=tf.string, 
                                   name='text'),
                    vectorizer_layer,
                    tf.keras.layers.Embedding(vocab_size, embedding_dim),
                    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units)),
                    tf.keras.layers.Dense(vocab_size, activation='softmax')])
    
    return vectorizer_layer, model

The subclassing method in TensorFlow is used for novel techniques mainly for research problems. The model I am building is pretty standard, but I wanted to use this opportunity play around with the subclassing methodology.

In the constructor each layer of the model is declared as an attribute of the object and the vectorizer layer is instantiated. The `call` method of the class is used to define the forward form of the model. Like in all Keras models, the forward form is all that is needed to be defined and Keras/Tensorflow handles computing the necessary information needed for [backpropegation](https://en.wikipedia.org/wiki/Backpropagation). The model consists of a vectorizer layer which converts the text which is made up of `seq_length` words into a `seq_length`-dimensional vector of integers that taken values between 1 and `vocab_size`. Next an embedding layer is applied, followed by a bi-directional GRU layer and softmax as the last layer to predict which of the `vocab_size` class the next word is.

The last method of the class is `get_wordmap`. This function returns the reverse dictionary that converts a integer representation of words back to its English version. This function is used for converting the output of the model back an English word.

Now, the model can be instantiated with 128 dimensional embedding layer and 64 unit GRU layer:

In [35]:
vectorizer, model = build_model(text=text, 
                                seq_length=10, 
                                vocab_size=15000, 
                                embedding_dim=128, 
                                units=64)

2023-05-25 17:27:08.047751: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Notice that for the vectorizer layer I have to pass the original text, `seq_length` and the `vocab_size` values to initialize that layer properly. 

Now we can compile the model:

In [36]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

Normally in a Tensorflow model using a [Sequential model](https://www.tensorflow.org/guide/keras/sequential_model) or the [Functional API](https://www.tensorflow.org/guide/keras/functional) once the model is compiled the [summary](https://keras.io/api/models/model/#summary-method) method can be used to return information on the model. However, compiling the model is not sufficient for using the [summary](https://keras.io/api/models/model/#summary-method) method in the subclassing API.


In [100]:
dataset = (tf.data.Dataset
             .from_tensor_slices((X, y))
             .shuffle(100000)
             .batch(128))

In [101]:
history = model.fit(dataset, epochs=25)

Epoch 1/25
9301/9301 [==============================] - 259s 28ms/step - loss: 4.5997
Epoch 2/25
9301/9301 [==============================] - 252s 27ms/step - loss: 4.5852
Epoch 3/25
9301/9301 [==============================] - 441s 47ms/step - loss: 4.5704
Epoch 4/25
9301/9301 [==============================] - 259s 28ms/step - loss: 4.5621
Epoch 5/25
9301/9301 [==============================] - 254s 27ms/step - loss: 4.5535
Epoch 6/25
9301/9301 [==============================] - 631s 68ms/step - loss: 4.5507
Epoch 7/25
9301/9301 [==============================] - 975s 105ms/step - loss: 4.5423
Epoch 8/25
9301/9301 [==============================] - 940s 101ms/step - loss: 4.5410
Epoch 9/25
9301/9301 [==============================] - 887s 95ms/step - loss: 4.5354
Epoch 10/25
9301/9301 [==============================] - 1551s 167ms/step - loss: 4.5295
Epoch 11/25
9301/9301 [==============================] - 247s 27ms/step - loss: 4.5348
Epoch 12/25
9301/9301 [=========================

In [102]:
model.predict([X[4]])

array([[9.7009258e-12, 9.8117747e-12, 4.6239831e-02, ..., 5.1395332e-09,
        1.2419777e-08, 9.6487384e-12]], dtype=float32)

In [103]:
X[0]

array(['of particular importance to south dakota are the farm policies'],
      dtype='<U100')

In [104]:
model.save("jfk_model")#, save_format="tf")

In [44]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_2 (TextV  (None, 10)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 10, 128)           1920000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 15000)             1935000   
                                                                 
Total params: 3,929,496
Trainable params: 3,929,496
Non-trainable params: 0
_________________________________________________________________


## Generating Text  <a class="anchor" id="fourth-bullet"></a>
-----------

In [97]:
model = tf.keras.models.load_model("jfk_model")

2023-05-22 21:36:37.025776: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-22 21:36:37.030131: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [105]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
reverse_word_map = dict(map(reversed, word_index.items()))

In [106]:
y_pred = model.predict(X[0])

In [107]:
str(reverse_word_map[np.argmax(y_pred[0])])

'of'

In [118]:
test = X[9342][0]

In [119]:
test

'food is increased and diversified agricultural production is essential both'

In [120]:
def next_words(input_str, n):
    final_str = ''
    for i in range(n):
        prediction = model.predict(np.array([input_str]), verbose=0)
        next_word = str(reverse_word_map[np.argmax(prediction[0])])
        final_str += next_word + ' ' 
        input_str += ' ' + next_word
        input_str = ' '.join(input_str.split(' ')[1:])
    return final_str

In [121]:
new_text = next_words(test, 4)

In [122]:
test + " " + new_text

'food is increased and diversified agricultural production is essential both and the interdependence of '

## Next Steps  <a class="anchor" id="fifth-bullet"></a>
-------------
